# Notebook

In [407]:
import math
import cmath
import yaml

## Book Keeping

In [408]:
MODEL = "3GBA132210"
VERBOSITY = 1

In [409]:
SWITCHING_FREQUENCY = 1000
BUS_VOLTAGE = 650
PEAK_CARRIER_VOLTAGE = 3.75
GEARBOX_GAIN = 10
LOAD_MOMENT_OF_INERTIA = 0.64
LOAD_FRICTION_COEFFICIENT = 0.28
SPEED_SENSOR_GAIN = 2 / 100
SPEED_RESPONSE_TIME = 1.2
SPEED_OVERSHOOT = 6

In [410]:
"""
SWITCHING_FREQUENCY = 1000
BUS_VOLTAGE = 650
PEAK_CARRIER_VOLTAGE = 3.25
GEARBOX_GAIN = 10
LOAD_MOMENT_OF_INERTIA = 24.5
LOAD_FRICTION_COEFFICIENT = 4.04
SPEED_SENSOR_GAIN = 2 / 100
SPEED_RESPONSE_TIME = 1.2
SPEED_OVERSHOOT = 6
"""

'\nSWITCHING_FREQUENCY = 1000\nBUS_VOLTAGE = 650\nPEAK_CARRIER_VOLTAGE = 3.25\nGEARBOX_GAIN = 10\nLOAD_MOMENT_OF_INERTIA = 24.5\nLOAD_FRICTION_COEFFICIENT = 4.04\nSPEED_SENSOR_GAIN = 2 / 100\nSPEED_RESPONSE_TIME = 1.2\nSPEED_OVERSHOOT = 6\n'

In [411]:
with open("../reports/" + MODEL + "_parameters.yaml", "r") as file:
    parameters = yaml.safe_load(file)

In [412]:
Kc = BUS_VOLTAGE / PEAK_CARRIER_VOLTAGE
Ts = 1 / SWITCHING_FREQUENCY

if VERBOSITY:
    print("Kc :: {0}".format(Kc))

Kc :: 173.33333333333334


In [413]:
Jr = parameters["Jr"]
Br = parameters["Br"]

Jeq = Jr + LOAD_MOMENT_OF_INERTIA / GEARBOX_GAIN**2
Beq = Br + LOAD_FRICTION_COEFFICIENT / GEARBOX_GAIN**2
Ksv = SPEED_SENSOR_GAIN * (30 / math.pi)
Ksv_ = Ksv / GEARBOX_GAIN

if VERBOSITY:
    print("Ksv :: {:.4e}".format(Ksv))
    print("Ksv' :: {:.4e}".format(Ksv_))
    print("Jeq :: {:.4e}".format(Jeq))
    print("Beq :: {:.4e}".format(Beq))

Ksv :: 1.9099e-01
Ksv' :: 1.9099e-02
Jeq :: 3.6100e-02
Beq :: 5.4131e-03


In [414]:
Vs = parameters["Vs"]
Ir = parameters["Ir"]
Rs = parameters["Rs"]
we = parameters["we"]
Ls = parameters["Ls"]
Lm = parameters["Lm"]
P = parameters["P"]
Rr = parameters["Rr"]

Kvf = (Vs - Ir * Rs) / (we * (1 + Ls / Lm))
Ksl = 3 * (P / 2)**2 * Kvf**2 * (1 / Rr)

if VERBOSITY:
    print("Kvf :: {:.4e}".format(Kvf))
    print("Ksl :: {:.4e}".format(Ksl))

Kvf :: 6.9288e-01
Ksl :: 8.5093e+00


In [415]:
Kma = (Ksl * Ksv_) / (Beq)
Tma = Jeq / Beq
zeta = -math.log(SPEED_OVERSHOOT / 100) / math.sqrt(math.pi**2 + math.log(SPEED_OVERSHOOT / 100)**2)
Tmf = SPEED_RESPONSE_TIME / 3

Kp = (((2 * Tma) / Tmf) - 1) / Kma
Ti = (Tmf**2 * zeta**2 * Kma) / Tma
tz = Kp * Ti

if VERBOSITY:
    print("Kp :: {:.4e}".format(Kp))
    print("Ti :: {:.4e}".format(Ti))
    print("tz :: {:.4e}".format(tz))

Kp :: 1.0774e+00
Ti :: 3.2057e-01
tz :: 3.4537e-01


In [416]:
Ir = parameters["Ir"]

Kvf_ = Kvf * (1 + Ls / Lm)
IrRs = Ir * Rs

if VERBOSITY:
    print("Kvf' :: {:.4e}".format(Kvf_))
    print("IrRs :: {:.4e}".format(IrRs))

Kvf' :: 7.1759e-01
IrRs :: 5.5036e+00


In [417]:
with open("../reports/" + MODEL + "_parameters.yaml", "a") as file:
    documents = yaml.dump({
            "Vcc": BUS_VOLTAGE,
            "Vtpp": PEAK_CARRIER_VOLTAGE,
            "fs": SWITCHING_FREQUENCY,
            "Kc": Kc,
            "Ksv": SPEED_SENSOR_GAIN,
            "Kg": GEARBOX_GAIN,
            "Bl": LOAD_FRICTION_COEFFICIENT,
            "Jl": LOAD_MOMENT_OF_INERTIA,
            "Kp": Kp,
            "Ti": Ti,
            "tz": tz,
            "Kvf_": Kvf_,
            "IrRs": IrRs,
            "Ksv_": Ksv_,
        }, 
        file
    )